We use Unsloth since it handles the conversion to GGUF for Raspberry Pi

In [ ]:

!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
!pip install --no-deps xformers trl peft accelerate bitsandbytes

  Cloning https://github.com/unslothai/unsloth.git to /tmp/pip-install-dcm07rl6/unsloth_7af5d47b0a614fee9c94de005bc889b7
  Running command git clone --filter=blob:none --quiet https://github.com/unslothai/unsloth.git /tmp/pip-install-dcm07rl6/unsloth_7af5d47b0a614fee9c94de005bc889b7
  Resolved https://github.com/unslothai/unsloth.git to commit ec1757c1a02175851146ff5f6ab2a26c8c863fc8
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.1/59.1 MB 18.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 506.8/506.8 kB 42.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.7/295.7 kB 32.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 180.7/180.7 kB 19.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.7/47.7 MB 18.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 149.8 MB/s eta 0:00:00

In [ ]:
# Step 2: Load Model
# unsloth: open-source library to make fine-tuning LLMs (vs. Hugging Face, unsloth 2x to 30x more faster, reduces VRAM by 60-80%)
from unsloth import FastLanguageModel
import torch
from datasets import Dataset
import pandas as pd

# Configuration
max_seq_length = 2048 # Context window -> model will remember 2048 tokens (1500 words) at a time
dtype = None  # data type for calculations, None allows Unsloth to auto-detect the GPU
load_in_4bit = True # Will use Quantization
# Quantization -> instead of loading model params as 16-bit floating point numbers, it compresses them into 4-bit integers
# So, the model tes up 4 times less RAM, to fit on free Google Colab GPUs

print("Loading Llama-3.2-3B-Instruct...")

"""
Unsloth -> Handles RoPE Scaling (Rotary Positional Embeddings) automatically;
           if you load a model designed for 4k context but ask for 8k, it
           mathematically stretches model capacity to handle it
FastLanguageModel.from_pretrained():
  - Acts as a specialized wrapper around Hugging Face standard loading methods, but
  with heavy optimizations during loading process
  - Why use this instead of AutoModelForCausalLM? It's the standard Hugging Face
  AutoModel class, it loads the standard, slow version of the model

Llama-3.2-3B-Instruct:
 - 3 Billion params
 - Instruct: already fine-tuned to follow chat instructions

Tokenizer: tool that converts text (strings) into numbers (tokens) that model understands
"""
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Llama-3.2-3B-Instruct",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)
"""
FastLanguageModel outputs the following:
  1. model: Actual Neural Network -> it's just a "Base Model"; it contains knowledge from
                              its original training, but it's read-only until we add
                              adapters
  2. tokenizer: Translator -> it converts human text into numbers that the model understands
"""

"""
Add LoRA adapters -> Lo-Rank Adapdation
Llama is like a giant 1,000-page textbook; Traditional Fine-Tuning implies rewriting the entire textbook to add new information. Slow and expensive
LoRA (Adapters) is like leaving the textbook exactly as it is and instead we add notes on specific pages with new updates
Slice of the model that sits on top of the frozen giant brain
"""
"""
get_peft_model(...):
  - instead of retraining all 3 billion params, we freeze main model and attach tiny trainable matrices (adapters)4
"""
model = FastLanguageModel.get_peft_model(
    model,
    r = 16,   # Rank
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,    # Determines how loudly the fine-tuned adapter speaks, alpha / r = 1 mean model blends new data perfectly with old data
    lora_dropout = 0,   # Dropout -> technique used to stop the model from memorizing data (overfitting), set to 0 since with LoRA, unecessary
    bias = "none",   # forbidden to touch the bias (starting point); It adds extra memory overhead and zero noticeable improvement in model quality with LoRA
    use_gradient_checkpointing = "unsloth",
    random_state = 3407,
    use_rslora = False,
    loftq_config = None,
)
"""
use_gradient_checpointing = "unsloth":
We do two passes when training a neural network: Forward pass (data flows through the model, layer by layer, to produce an output)
and backward pass (the error is sent back through the model to update the weights)
To do backward pass, GPU needs to remember exactly what the data looked like at every single layer during forward process. These are called activations
and storing these activations for a large model eats up GB of VRAM
Solution: we use checkpointing. It tells GPU to don't memorize everything, just memorize a few checkpoints.
We save a lot of memory, but training is slower
"""

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
Loading Llama-3.2-3B-Instruct...
==((====))==  Unsloth 2026.1.2: Fast Llama patching. Transformers: 4.57.3.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.9.0+cu126. CUDA: 7.5. CUDA Toolkit: 12.6. Triton: 3.5.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.33.post2. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/2.35G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/234 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/454 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

chat_template.jinja: 0.00B [00:00, ?B/s]

Unsloth 2026.1.2 patched 28 layers with 28 QKV layers, 28 O layers and 28 MLP layers.


'\nuse_gradient_checpointing = "unsloth":\nWe do two passes when training a neural network: Forward pass (data flows through the model, layer by layer, to produce an output)\nand backward pass (the error is sent back through the model to update the weights)\nTo do backward pass, GPU needs to remember exactly what the data looked like at every single layer during forward process. These are called activations\nand storing these activations for a large model eats up GB of VRAM\nSolution: we use checkpointing. It tells GPU to don\'t memorize everything, just memorize a few checkpoints.\nWe save a lot of memory, but training is slower\n'

In [ ]:
!pip install HuggingFace

In [ ]:
from google.colab import files
print("Please select your CSV file from your computer...")
uploaded = files.upload()

Please select your CSV file from your computer...


Saving github_startups.csv to github_startups.csv


In [ ]:
"""
STEP 3: prepare data
Need to turn csv file into a specific "Prompt Format"
Model needs to see: Instruction + Input + Expected Output
"""
alpaca_prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{}

### Input:
{}

### Response:
{}"""

try:
  df = pd.read_csv("github_startups.csv")
  print("Loaded github_startups.csv")
except:
  print("github_startups.csv not found. Generating dummy data for demonstration...")
  data = {
      'description': [
          "We are building a decentralized exchange on Solana using Rust. Our TPS is 50k and we use Serum for liquidity.",
          "An AI tool that uses PyTorch to detect cancer in X-rays.",
          "A React-based dashboard for tracking carbon emissions."
      ] * 10
  }
  df = pd.DataFrame(data)

# Because we don't have real "Investment Memos", we will synthetically format the
# output to teach the model the structure we want
def format_dataset(row):
  instruction = "Analyze this startup description and generate a structured Investment Memo."
  input_text = row['description']

  # We create a synthetic "target" (This is what the model tries to learn)
  # We force the model to learn this specific Markdown structure
  output_text = f"""
**Problem:** Need to infer from text (Synthesized).
**Solution:** Solution based on {input_text[:20]}...
**Tech Stack:** Extracted relevant tech.
**Verdict:** AUTOMATED ANALYSIS
  """

  # Combine into full prompt
  text = alpaca_prompt.format(instruction, input_text, output_text) + tokenizer.eos_token
  return {"text": text}

# Convert to HuggingFace Dataset
dataset = Dataset.from_pandas(df)
dataset = dataset.map(format_dataset, batched=False)

print("Sample Data fed to model:")
print(dataset[0]['text'])

Loaded github_startups.csv


Map:   0%|          | 0/100 [00:00<?, ? examples/s]

Sample Data fed to model:
Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
Analyze this startup description and generate a structured Investment Memo.

### Input:
An Open Source Machine Learning Framework for Everyone

### Response:

**Problem:** Need to infer from text (Synthesized).
**Solution:** Solution based on An Open Source Machi...
**Tech Stack:** Extracted relevant tech.
**Verdict:** AUTOMATED ANALYSIS
  <|eot_id|>


In [ ]:
# Step 4: Train the model (Fine-tuning)

# SFT: Supervised Fine-Tuning
from trl import SFTTrainer
from transformers import TrainingArguments

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = False,    # Can speed up training for short sequences
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        max_steps = 60,    # Small number for demo, increase to 300+ for real results
        learning_rate = 2e-4,
        fp16 = not torch.cuda.is_bf16_supported(),
        bf16 = torch.cuda.is_bf16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
    ),
)

print("Starting training...")
trainer_stats = trainer.train()
print("Training complete!")

Unsloth: Tokenizing ["text"] (num_proc=4):   0%|          | 0/100 [00:00<?, ? examples/s]

The model is already on multiple devices. Skipping the move to device specified in `args`.


Starting training...


==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 100 | Num Epochs = 5 | Total steps = 60
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 4 x 1) = 8
 "-____-"     Trainable parameters = 24,313,856 of 3,237,063,680 (0.75% trained)
wandb: (1) Create a W&B account
wandb: (2) Use an existing W&B account
wandb: (3) Don't visualize my results
wandb: Enter your choice:

 3


...

Step,Training Loss
1,4.359500
2,4.294000
3,4.203400
4,4.006600
5,3.741500
6,3.533000
7,3.142500
8,2.744500
9,2.580100
10,2.487100


Unsloth: Will smartly offload gradients to save VRAM!


wandb: WARNING URL not available in offline run


train/epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇█████
train/global_step,▁▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇██
train/grad_norm,▅▅▅▅▆▆▅▇▇█▇▅▄▄▂▂▂▂▃▂▄▅▅▅▄▂▁▁▁▁▁▁▁▁▂▁▁▁▁▂
train/learning_rate,▁▂▄▇██▇▇▇▇▇▇▇▆▆▆▆▆▆▅▅▅▅▅▅▄▄▃▃▃▃▃▃▂▂▂▂▂▁▁
train/loss,███▇▆▅▄▃▂▃▂▂▁▂▂▂▂▂▂▂▁▁▁▁▂▁▁▁▁▁▁▁▁▁▂▁▁▁▁▁
total_flos,1012379993677824.0
train/epoch,4.64
train/global_step,60
train/grad_norm,0.51938
train/learning_rate,0.0
train/loss,0.5385


Training complete!


In [ ]:
# Step 5: Export to GGUF (For Raspberry Pi)
# Converts the model to a format the Pi can run

print("Saving to GGUF format...")
model.save_pretrained_gguf("model_pi", tokenizer, quantization_method = "q4_k_m")
print("Download the model.pi folder")

Saving to GGUF format...
Unsloth: Merging model weights to 16-bit format...


config.json:   0%|          | 0.00/890 [00:00<?, ?B/s]

Found HuggingFace hub cache directory: /root/.cache/huggingface/hub


model.safetensors.index.json: 0.00B [00:00, ?B/s]

Checking cache directory for required files...
Cache check failed: model-00001-of-00002.safetensors not found in local cache.
Not all required files found in cache. Will proceed with downloading.
Checking cache directory for required files...
Cache check failed: tokenizer.model not found in local cache.
Not all required files found in cache. Will proceed with downloading.


...

model-00002-of-00002.safetensors:   0%|          | 0.00/1.46G [00:00<?, ?B/s]

Unsloth: Preparing safetensor model files: 100%|██████████| 2/2 [02:05<00:00, 62.52s/it]


Note: tokenizer.model not found (this is OK for non-SentencePiece models)


Unsloth: Merging weights into 16bit: 100%|██████████| 2/2 [03:08<00:00, 94.18s/it]


Unsloth: Merge process complete. Saved to `/content/model_pi`
Unsloth: Converting to GGUF format...
==((====))==  Unsloth: Conversion from HF to GGUF information
   \\   /|    [0] Installing llama.cpp might take 3 minutes.
O^O/ \_/ \    [1] Converting HF to GGUF f16 might take 3 minutes.
\        /    [2] Converting GGUF f16 to ['q4_k_m'] might take 10 minutes each.
 "-____-"     In total, you will have to wait at least 16 minutes.

Unsloth: Installing llama.cpp. This might take 3 minutes...
Unsloth: Updating system package directories
Unsloth: All required system packages already installed!
Unsloth: Install llama.cpp and building - please wait 1 to 3 minutes
Unsloth: Cloning llama.cpp repository
Unsloth: Install GGUF and other packages
Unsloth: Successfully installed llama.cpp!
Unsloth: Preparing converter script...
Unsloth: [1] Converting model into f16 GGUF format.
This might take 3 minutes...
Unsloth: Initial conversion completed! Files: ['llama-3.2-3b-instruct.F16.gguf']
Unsloth: 

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
import shutil

source_folder = "/content"
destination_folder = "/content/drive/MyDrive/Startup_Model"

# Create the destination folder in your Drive
if not os.path.exists(destination_folder):
    os.makedirs(destination_folder)

# Find the file and copy it
try:
    files = [f for f in os.listdir(source_folder) if f.endswith('.gguf')]
    if files:
        filename = files[0]
        src_path = os.path.join(source_folder, filename)
        dst_path = os.path.join(destination_folder, filename)

        print(f"Copying {filename} to Google Drive...")
        print("This may take 1-2 minutes...")

        shutil.copy2(src_path, dst_path)
        print(f"SUCCESS! File copied to: {dst_path}")
        print("Go to drive.google.com now to download it.")
    else:
        print("ERROR: No .gguf file found in model_pi. Did the saving step finish?")
except Exception as e:
  print(f"Error: {e}")

Copying llama-3.2-3b-instruct.Q4_K_M.gguf to Google Drive...
This may take 1-2 minutes...
SUCCESS! File copied to: /content/drive/MyDrive/Startup_Model/llama-3.2-3b-instruct.Q4_K_M.gguf
Go to drive.google.com now to download it.
